In [83]:
import requests
from bs4 import BeautifulSoup

import random
import re
import pandas as pd
import numpy as np

from datetime import datetime


In [84]:
wiki_link = 'https://en.wikipedia.org/wiki/2025_in_hip-hop'

headers = {
            'User-Agent': random.choice([
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)',
                'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0)',
            ])
        }

html_text = requests.get(wiki_link, headers=headers).text
soup = BeautifulSoup(html_text, 'lxml')

In [85]:
wiki_scrape = soup.select('div.mw-heading.mw-heading3,table.wikitable')
#now we want to associate each table with a date. could do {Date:Table}

In [86]:
wiki_scrape[3].find('h3').get('id')

'January_2'

In [87]:
data = {}
prev_header = None

for n in wiki_scrape:
    if n.get('class') == ['mw-heading', 'mw-heading3']:
        prev_header = n.find('h3').get('id')
    elif n.get('class') == ['wikitable']:
        data[prev_header] = pd.read_html(str(n))[0]

# Data in pandas

In [95]:
df_lst = []
for month, table in data.items():
    table['Month'] = re.sub('[^a-zA-Z]+', '', month)
    df_lst.append(table)

In [96]:
albums_release_data = pd.concat(df_lst)#.to_csv('out.csv', index = False)

In [97]:
albums_release_data = albums_release_data[['Artist(s)','Album','Record label(s)', 'Month', 'Day']]

In [98]:
albums_release_data['Month'] = albums_release_data['Month'].replace('UnscheduledandTBA', np.nan)

In [99]:
albums_release_data['Day'] = albums_release_data['Day'].replace('TBA', np.nan)

In [100]:
print(albums_release_data.to_string())
#how should multiple artists/record labels be handled? list?
#create new table for any list when data modeling

                                       Artist(s)                                                         Album                                                         Record label(s)     Month   Day
0                                      03 Greedo                                2025: The Streetz is Over Wiit                                                       Self-released[15]   January     1
1                                    Duwap Kaine                                  Nobody Is Coming To Save You                                                                 TAF[16]   January     1
2                               Edward Skeletrix                                                  Museum Music                                                           Self-released   January     1
3                        Boldy James & RichGains                                    Murder During Drug Traffic                                                       Self-released[17]   January     3
4    

In [102]:
albums_release_data

,Artist(s),Album,Record label(s),Month,Day
0,03 Greedo,2025: The Streetz is Over Wiit,Self-released[15],January,1
1,Duwap Kaine,Nobody Is Coming To Save You,TAF[16],January,1
2,Edward Skeletrix,Museum Music,Self-released,January,1
3,Boldy James & RichGains,Murder During Drug Traffic,Self-released[17],January,3
4,Jon Connor,Food for the Soul,Self-released[18],January,3
...,...,...,...,...,...
90,Xzibit,Kingmaker,Greenback Records,NaN,NaN
91,Young Thug,TBA,"YSL, 300, Atlantic",NaN,NaN
92,Your Old Droog & Madlib,TBA,Madlib Invazion,NaN,NaN
93,¥$,Vultures 3,YZY,NaN,NaN


In [103]:
def scrape(link):
    headers = {
            'User-Agent': random.choice([
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)',
                'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0)',
            ])
        }
    
    html_text = requests.get(link, headers=headers).text

    return BeautifulSoup(html_text, 'lxml')

def get_top_monthly_listeners():
    kworb_link = 'https://kworb.net/spotify/listeners.html'

    soup = scrape(kworb_link)
    table = soup.select('tbody')[0].select('tr')

    data = {}
    for rank, row in enumerate(table[:500]):
        artist = row.select('td')[0].text.encode("latin-1").decode("utf-8")
        data[artist] = {}
        data[artist]["ranking"] = int(rank + 1)
        data[artist]["monthly_listners"] = int(str.replace(row.select('td')[1].text, ",", ""))

    return data

In [104]:
top_artists = get_top_monthly_listeners()

In [108]:
albums_release_data[albums_release_data['Artist(s)'].isin(top_artists.keys())]

,Artist(s),Album,Record label(s),Month,Day
5,Lil Baby,WHAM,Quality Control Music[19],January,3
7,Bad Bunny,Debí Tirar Más Fotos,Rimas[20],January,5
18,Mac Miller,Balloonerism,Warner[30],January,17
31,Central Cee,Can't Rush Greatness,Columbia[39],January,24
53,The Weeknd,Hurry Up Tomorrow,"XO, Republic[58]",January,31
8,Tyga,NSFW,Last Kings[64],February,7
14,Tory Lanez,Peterson,One Umbrella[106],March,7
15,YoungBoy Never Broke Again,More Leaks,"Never Broke Again, Motown",March,7
5,Playboi Carti,I Am Music,"AWGE, Interscope Records",March,14
10,Lil Durk,Deep Thoughts,Alamo,March,28
